# Dynamics of Politicians

The model that we discussed in this post and the previous one is a pretty basic dynamic programming problem. In the context of econ, the framework of being elected vs. not elected is pretty similar to the job search question. Just as a brief recap, the idea of dynamic programming is to take a difficult multi-step problem and break them down into sequence of similar decision steps over time. 

In the politician problem that we have been discussing, the key is to identify how to break down this complicated issue of what decisions a politican makes and I've settled on primarily focussing on channels of affecting the election outcome probabilities. 

In [ ]:
# Load in some helpful packages
import numpy as np
import matplotlib.pyplot as plt


The cell below is probably the most important one when it comes to making something interesting. Doing the value iteration solving itself is not all that exciting once you know how to do it and have done it many times before, but the value and relative values of the parameters below is what drives the insights we can make. There are many choices to make: what should the moral compasses look like for each the politician, constituency, and political party, what are the relative importance of consumption and voting in forming utility or elections spendings and voting in the transition probabilities, what functions/distributions should we use for the utility and transition probabilities. In an unestimated model, each of of these decisions have a significant impact on what comes out of the model.

In [ ]:
# Initialize model parameters
political_spectrum_size = 2
constituency_moral_compass = np.array([0.6, 0.6])
party_moral_compass = np.ones(political_spectrum_size)


# Utility weights, these determine how important a politician's morality is relative to consuming money 
phi_c = 1.0
phi_v = 1.0

# temporal discount factor
beta = 0.95

# voted-in wage, moral compass, utility function, and transition probability distribution
w_i = 10
moral_compass = np.ones(political_spectrum_size)
u_i = lambda c,v: np.exp(-(phi_c * np.linalg.norm(v - moral_compass))/(phi_v * c))
draw_transition_i = lambda zeta: np.random.beta()

# voted-out wage, utility function, and transition probability distribution
w_o = 5
u_o = lambda c: np.exp(-(phi_c)/(c))
draw_transition_o = lambda zeta: np.random.beta(a = zeta, b = 7.5)

In [ ]:
# Initialize the state space

